In [6]:
import tensorflow as tf
tf.test.is_gpu_available()

True

# What about a GAN + Self correcting U-Net ? That would make for a cool architecture
# Following CGAN -> adding a 1-hot vector encoding of the label to the training data
# Simulated Annealing?
# Generator -> VAE -> Discriminator?
# What about feeding in a dicriminator's confidence level as a temperature during the autoregressive? Inverse confidence?
# What about a 3 dimensional GAN?

In [7]:
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.utils import plot_model
from keras import backend as K
from tensorflow.python.ops import math_ops

def built_in_softmax_kl_loss(target, output):
    target = K.flatten(target)
    output = K.flatten(output)
    
    target = target / K.sum(target)
    output = K.softmax(output)
    return keras.losses.kullback_leibler_divergence(target, output)

keras.losses.built_in_softmax_kl_loss = built_in_softmax_kl_loss
 
def unet_model(input_size=(28, 28, 1), n_filters_start=16, growth_factor=2,
               upconv=False):
    droprate=0.1
    n_filters = n_filters_start
    inputs = Input(input_size)
    conv_first = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(inputs)
    conv_first = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv_first)
    pool_first = MaxPooling2D(pool_size=(2, 2))(conv_first)

    prev_pool = pool_first
    hidden_layers = []
    for _ in range(1):
        n_filters *= growth_factor
        pool = BatchNormalization()(prev_pool)
        conv = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(pool)
        conv = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv)
        pool = MaxPooling2D(pool_size=(2, 2))(conv)
        pool = Dropout(droprate)(pool)
        prev_pool = pool
        hidden_layers.append(conv)
 
    n_filters *= growth_factor
    conv_mid = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(prev_pool)
    conv_mid = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv_mid)
 
    n_filters //= growth_factor
    if upconv:
        up_first = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv_mid), hidden_layers[-1]])
    else:
        up_first = concatenate([UpSampling2D(size=(2, 2))(conv_mid), hidden_layers[-1]])
    up_first = BatchNormalization()(up_first)
    conv_mid_2 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up_first)
    conv_mid_2 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv_mid_2)
    conv_mid_2 = Dropout(droprate)(conv_mid_2)

    prev_conv = conv_mid_2
    for i in range(0):
        n_filters //= growth_factor
        if upconv:
            up = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(prev_conv), hidden_layers[-i-2]])
        else:
            up = concatenate([UpSampling2D(size=(2, 2))(prev_conv), hidden_layers[-i-2]])
        up = BatchNormalization()(up)
        conv = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up)
        conv = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv)
        conv = Dropout(droprate)(conv)
        prev_conv = conv
 
    n_filters //= growth_factor
    if upconv:
        up_last = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv_mid_2), conv_first])
    else:
        up_last = concatenate([UpSampling2D(size=(2, 2))(conv_mid_2), conv_first])
    conv_last = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up_last)
    conv_last = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv_last)
 
    conv_out = Conv2D(1, 1, activation='linear')(conv_last)
 
    model = Model(inputs=inputs, outputs=conv_out)
    # model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer=Adam(), loss=built_in_softmax_kl_loss, metrics=['accuracy'])
    model.summary()
    return model

In [58]:
import mnist
import scipy.misc
from PIL import Image
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from copy import deepcopy

def create_image(image, name, image_shape=(28, 28)):
    img_arr = deepcopy(image.reshape(image_shape)).astype(np.uint8)
    # print("img shape: {}. img sum: {}".format(img_arr.shape, img_arr.sum()))
    # print(img_arr)
    img_arr[img_arr > 0] = 255
    # pprint(img_arr)
    # print("img shape: {}. img sum: {}".format(img_arr.shape, img_arr.sum()))
    img = Image.fromarray(img_arr.astype(np.uint8), 'L')
    img.save(name)
    return img

images = mnist.train_images()
images[images > 0] = 1
# images = images.reshape(images.shape[0], -1)
print(images.shape)

labels = mnist.train_labels()
n_labels = np.max(labels) + 1
labels = np.eye(n_labels)[labels]
print(labels.shape)

create_image(images[0], 'my.png')
print(labels[0])

# images = images[:1000, :, :]
# print(images[0].shape)
# pprint(images[0])
# img = Image.fromarray(images[0], 'L')
# img.save('my.png')
# img.show()

image_shape = np.expand_dims(images[0], axis=-1).shape 
# print(image_shape)

(60000, 28, 28)
(60000, 10)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [14]:
# model = unet_model(input_size=image_shape)

In [15]:
def discriminator():
    pass

In [16]:
from copy import deepcopy
import math
import itertools
def mask_image(image):
    image = deepcopy(image)
    sampling_percentage = np.random.uniform(0, 100)
    non_zero = np.nonzero(image)
    mask1 = np.full(len(non_zero[0]), False)
    mask1[:math.floor(len(non_zero[0]) * (sampling_percentage/100))] = True
    np.random.shuffle(mask1)
    # pprint(mask1)
    r1 = list(itertools.compress(non_zero[0], mask1))
    c1 = list(itertools.compress(non_zero[1], mask1))
    output_image = deepcopy(image)
    output_image[r1,c1] = 0
    return output_image

def mask_image_with_noise(image):
    image = deepcopy(image)
    sampling_percentage_mask = np.random.uniform(0, 100)
    sampling_percentage_noise = np.random.uniform(0, 20)
    non_zero = np.nonzero(image)
    zeroes = np.nonzero(image == 0)
    mask = np.full(len(non_zero[0]), False)
    noise = np.full(len(zeroes[0]), False) 
    mask[:math.floor(len(non_zero[0]) * (sampling_percentage_mask/100))] = True
    noise[:math.floor(len(non_zero[0]) * (sampling_percentage_noise/100))] = True
    np.random.shuffle(mask)
    np.random.shuffle(noise)
    # pprint(mask1)
    output_image = deepcopy(image)
    r1 = list(itertools.compress(non_zero[0], mask))
    c1 = list(itertools.compress(non_zero[1], mask))
    r2 = list(itertools.compress(zeroes[0], noise))
    c2 = list(itertools.compress(zeroes[1], noise))
    output_image[r1,c1] = 0
    output_image[r2,c2] = 1
    return output_image

class ImageGenerator(keras.utils.Sequence):
    def __init__(self, sample_list, image_shape, batch_size, samples_per_data_item, seed=None):
        print("sample_list: {}".format(len(sample_list)))
        self.sample_list = sample_list
        self.image_shape = image_shape
        self.batch_size = batch_size
        self.samples_per_data_item = samples_per_data_item
        self.training_input = []
        self.training_target = []
        self.training_original = []
        self.sample_index = 0
        self.seed = seed
        if self.seed is not None:
            np.random.seed(self.seed)

    def generate_training_pairs(self):
        '''
        Generates Training Pairs till @training_input / @training_target have @batch_size files.
        '''
        while len(self.training_input) < self.batch_size:
            original_image = self.sample_list[self.sample_index]
            original_image = original_image.reshape(self.image_shape)
            original_image[original_image > 0] = 1
            self.sample_index = (self.sample_index + 1) % len(self.sample_list)
            # print("sample_list length: {}. sample_index: {}".format(
            #     len(self.sample_list), self.sample_index))
            try:
                # augment by adding and removing random values in the array

                # Add random values
                for _ in range(self.samples_per_data_item):
                    input_image = mask_image_with_noise(original_image)

                    xor_target = np.logical_xor(input_image, original_image)
                    input_image = input_image.astype(np.float32)
                    xor_target = xor_target.astype(np.float32)
                    original_image = original_image.astype(np.float32)
                    self.training_input.append(deepcopy(input_image))
                    self.training_original.append(deepcopy(original_image))
                    self.training_target.append(deepcopy(xor_target))


            except Exception as e:
                print('Error generating input and target pair')
                traceback.print_exc()

    def save_image(self, img_arr, img_name):
        print(img_name)
        print("img shape: {}. img sum: {}".format(img_arr.shape, img_arr.sum()))
        img_arr = img_arr[:, :, 0]
        print("img shape: {}. img sum: {}".format(img_arr.shape, img_arr.sum()))
        #pprint(img_arr)
        img_arr[img_arr != 0] = 255
        #pprint(img_arr)
        print("img shape: {}. img sum: {}".format(img_arr.shape, img_arr.sum()))
        img = Image.fromarray(img_arr.astype(np.uint8), 'L')
        img.save(img_name)

    def get_random_training_pair(self):
        import random
        self.generate_training_pairs()
        index = random.randrange(0, len(self.training_input))
        self.save_image(deepcopy(self.training_input[index]), 'training_input.png')
        self.save_image(deepcopy(self.training_target[index]), 'training_target.png')
        self.save_image(deepcopy(self.training_original[index]), 'training_original.png')

    def generate_validation_samples(self):
        old_batch_size = self.batch_size
        self.batch_size = len(self.sample_list) * self.samples_per_data_item
        self.generate_training_pairs()
        training_input = np.asarray(self.training_input[:self.batch_size])
        training_target = np.asarray(self.training_target[:self.batch_size])
        self.batch_size = old_batch_size
        return training_input, training_target

    def __getitem__(self, index):
        '''Generates 1 batch of data'''
        self.generate_training_pairs()
        training_input = np.asarray(self.training_input[:self.batch_size])
        training_target = np.asarray(self.training_target[:self.batch_size])
        self.training_input = self.training_input[self.batch_size:]
        self.training_target = self.training_target[self.batch_size:]
        # print("training input sum: {}. target sum: {}".format(training_input.sum(), training_target.sum()))
        return training_input, training_target

    def __len__(self):
        '''Number of batches / epoch'''
        # print("sample_list: {}. samples_per_data_item: {}, batch size: {}".
        #       format(len(self.sample_list), self.samples_per_data_item,
        #              self.batch_size))
        samples_to_generate = int(
            (len(self.sample_list) * self.samples_per_data_item) /
            self.batch_size)
        # print("samples to generate: {}".format(samples_to_generate))
        return samples_to_generate
    
    def on_epoch_end(self):
        if self.seed is not None:
            np.random.seed(self.seed)


In [17]:
batch_size = 32
samples_per_data_item = 4

split = 0.8

training_samples = images[:int(len(images) * split)]
validation_samples = images[int(len(images) * split):]

print("training samples: {}. validation samples: {}".format(len(training_samples), len(validation_samples)))

steps_per_epoch = int(len(training_samples) * samples_per_data_item / batch_size)

# pprint(training_samples[0])

training_generator = ImageGenerator(
    sample_list=training_samples,
    image_shape=image_shape,
    batch_size=batch_size,
    samples_per_data_item=samples_per_data_item)

validation_generator = ImageGenerator(
    sample_list=validation_samples,
    image_shape=image_shape,
    batch_size=batch_size,
    samples_per_data_item=samples_per_data_item,
    seed=7)

# validation_data = validation_generator.generate_validation_samples()

# print("validation data input and target shape: {}".format(validation_data[0].shape))
validation_generator.get_random_training_pair()




training samples: 48000. validation samples: 12000
sample_list: 48000
sample_list: 12000
training_input.png
img shape: (28, 28, 1). img sum: 77.0
img shape: (28, 28). img sum: 77.0
img shape: (28, 28). img sum: 19635.0
training_target.png
img shape: (28, 28, 1). img sum: 185.0
img shape: (28, 28). img sum: 185.0
img shape: (28, 28). img sum: 47175.0
training_original.png
img shape: (28, 28, 1). img sum: 216.0
img shape: (28, 28). img sum: 216.0
img shape: (28, 28). img sum: 55080.0


In [18]:
#validation_data = validation_data_generator.generate_validation_pairs()
#print("Done generating validation data", getsizeof(validation_data), validation_data[0].nbytes)

#tensorflow_log_dir = cfg.tensorboard_log_dir + cfg.experiment_name
# checkpoint_path = cfg.checkpoint_root + cfg.experiment_name + "-weights-epoch:{epoch:02d}-{val_loss:.2f}.hdf5"
# checkpoint_path = cfg.checkpoint_root + cfg.experiment_name + "-best-model-epoch:{epoch:04d}.hdf5"

# callbacks = keras_traning_callbacks(tensorflow_log_dir,
                                      # checkpoint_path,
                                      # tag=cfg.experiment_name,
                                      # n_timesteps=n_timesteps,
                                      # steps_per_log=steps_per_epoch,
                                      # steps_per_save=steps_per_epoch)




In [19]:
import os
# model = unet_model(input_size=(28, 28, 1))
model = keras.models.load_model('sc-model.hdf5')

#history = model.fit_generator(
#    training_generator,
    # validation_data=validation_data,

#zeroed = deepcopy(training_samples)
#zeroed[True] = 0
#pprint(zeroed)
#pprint(training_samples)

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(F'sc-model.hdf5'),
    monitor='val_loss',
    save_weights_only=False,
    verbose=1,
    mode='min',
    save_best_only=True)

tensorboard_callback = keras.callbacks.TensorBoard(log_dir='./logs', update_freq='epoch')

if False:
    history = model.fit(
        training_generator,
        validation_data=validation_generator,
        verbose=1,
        shuffle=True,
        steps_per_epoch=steps_per_epoch,
        epochs=200,
        callbacks=[model_checkpoint_callback, tensorboard_callback])
    #epochs=cfg.epochs,
    #callbacks=callbacks)
# model.save("sc-model.hdf5")

Epoch 1/200


KeyboardInterrupt: 

In [21]:
# "Loss"
if False:
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

# Inference

In [22]:
# Load the best model
model = keras.models.load_model('sc-model.hdf5')

# input_image = mask_image_with_noise(images[0])
input_image = np.random.rand(28, 28)
input_image[input_image >= 0.5] = 1
input_image[input_image < 0.5] = 0
input_image = input_image.astype(np.float32)
# pprint(input_image)
input_image = np.expand_dims(input_image, 0)
input_image = np.expand_dims(input_image, -1)
print(input_image.shape)

create_image(input_image, "input.png")


working_image = deepcopy(input_image)


(1, 28, 28, 1)
img shape: (28, 28). img sum: 406
img shape: (28, 28). img sum: 103530


In [74]:
import os
import shutil
import time

def inference(model, input_image, directory, iterations, temp_start=2, temp_end=0.5):
    create_image(input_image, "{}/input.png".format(directory))

    temperatures = np.linspace(temp_end, temp_start, num=iterations)[::-1]
    # pprint(temperatures)
    
    working_image = deepcopy(input_image)
    for i in range(iterations):
        temp = temperatures[i]
        predictions = model.predict(working_image)
        predictions = predictions.flatten()
        predictions = np.exp(predictions / temp)
        predictions = predictions / np.sum(predictions)
        # print(predictions)
        indices = np.arange(predictions.shape[0])
        index = np.random.choice(indices, p=predictions)
        working_image = working_image.flatten()
        if working_image[index] == 1:
            working_image[index] = 0
        elif working_image[index] == 0:
            working_image[index] = 1
        else:
            print(working_image[index])
        working_image = np.reshape(working_image, [1, *image_shape])
        if i % 100 == 0:
            create_image(working_image, os.path.join(directory, "working_{}.png".format(i)))

    print(working_image.shape)
    img = create_image(working_image, os.path.join(directory, "final.png"))
    return img



In [77]:
images = []

sample_sqrt = 5
for i in range(sample_sqrt**2):
    directory = "images_{}".format(i * j)
    os.makedirs(directory, exist_ok=True)

    input_image = np.random.rand(28, 28)
    input_image[input_image >= 0.5] = 1
    input_image[input_image < 0.5] = 0
    input_image = input_image.astype(np.float32)
    # pprint(input_image)
    input_image = np.expand_dims(input_image, 0)
    input_image = np.expand_dims(input_image, -1)

    images.append(inference(model, input_image, directory, 2000, temp_start=2, temp_end=0.5))
    
final_im = Image.new('RGB', (28 * sample_sqrt, 28 * sample_sqrt))

y_offset = 0
for i in range(sample_sqrt):
    x_offset = 0
    new_im = Image.new('RGB', (28 * sample_sqrt, 28))
    for j in range(sample_sqrt):
        im = images[(i * sample_sqrt) + j]
        new_im.paste(im, (x_offset, 0))
        x_offset += 28
    final_im.paste(new_im, (0, y_offset))
    y_offset += 28
    
final_im.save('final.png')

(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)
(1, 28, 28, 1)


# New Section